In [ ]:
%matplotlib inline

import healpy as hp
import os,sys,glob,time,pickle
import matplotlib.pyplot as plt
import numpy as np

# Specific qubic modules
from pysimulators import FitsArray
from qubic.lib import QskySim as qss
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Qsamplings import equ2gal
from qubic.data import PATH as data_dir
print(data_dir)

plt.rcParams['figure.figsize'] =(16, 10)
plt.rcParams['font.size'] =15
plt.rcParams['image.cmap'] = 'jet'

This notebook is a demo for using the QUBIC FastSimulator which is a shortcut to produce QUBIC noise maps without having to go through the long and CPU greedy process of QUBIC Map-Making on supercomputers.

The FastSimulator reproduces the noise structure that has been measured so far in simulations:
- Noise RMS profile as a function of coverage
- Noise spatial correlation that is responsible for lower $\Delta C_\ell$ at low $\ell$ and is a key aspect of the QUBIC nosie structure, originating from noise anticorrelation at the scale of the distance between two synthesized beam peaks.
- Noise correlation (mostly anticorrelation) sub-bands reconstructed when performing spectro-imaging.

# Choose between TD and FI, 150 and 220GHz 




In [ ]:
#### FI220, FI150, TD150
config = 'FI220' 

### Read some stuff
# Read dictionary chosen
d = qubicDict()
d.read_from_file('pipeline_demo.dict')

d['nside'] = 256
center = equ2gal(d['RA_center'], d['DEC_center'])

# Frequency wide band, 150 or 220 GHz
d['filter_nu'] = int(config[-3:]) * 1e9
print(d['filter_nu'])

First, you need to instanciate a QubicSkySim (qss) object where the PYSM3 model can be specified (see doc/QubicSkySim/ notebooks for documentation and examples)

Then one can retrieve detailed approximations of the Qubic reconstructed sub-bands with some random noise realization with the correct properties:
- Noise profile as a function of coverage
- Spatial noise correlations
- subband-subband noise covariance
All these features have been fit on Qubic End-To-End (TOD to maps) simulations performed at NERSC or similar supercomputer.

The code returns the reconstructed maps with noise, the noiseless maps, the noise maps and the coverage map. Note that it can also use an alternative coverage than you provide as an input in the keyword coverage.

The sky map is first constructed with infinite resolution at ech of the "input sub frequencies" of the dictionary (d['nf_sub']) and then convolved at each of the resolutions of each frequencies. Then it is averaged oved the reconstructed frequencies (d['nf_recon]) in such a. way that is mimics Qubic End-To-End reconstruction as much as possible.

#### Very important remark:
When simulating with Spatial noise correlations, the RMS of the noise maps that are returned is **not at all** the expected one, the map however has the correct noise Cl power spectrum and should be used as it is (no renormalization of the noise to match the intended RMS) for Power Spectrum studies. If one is rather interested in map-space studies, and therefore is likely to need to rely on the RMS of the pixels at some point for statistical putposes, it is then important to set the keyword 'spatial_noise' to False in the call to the method get_partial_sky_maps_withnoise() of QubicSkySim.

### First Example: CMB Only, a single noise realization, no spatial_noise
In this case the code returns the number of sub-bands from the input dictionary (here 1). The maps are centered on the Galactic Center. Provide your own coverage map centered elsewhere (optional argument) for another target...

In [ ]:
#### One can easily change the number of sub-bands by updating the dictionary
nbands = 3
d['nf_recon'] = nbands
d['nf_sub'] = nbands

##### QubicSkySim instanciation
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)

##### Getting FastSimulator output maps
maps_qubic, maps_noiseless, maps_noise, coverage = Qubic_sky.get_partial_sky_maps_withnoise(spatial_noise=False)


################################# Displaying Results ##########################################
### Coverage map
hp.mollview(coverage, title='Coverage Map', sub=(2,2,1))
hp.gnomview(coverage, reso=25, title='Coverage Map', sub=(2,2,2))

### Maps per sub-bands
stn = ['I', 'Q', 'U']
plt.figure()
for i in range(d['nf_recon']):
    for s in range(3):
        hp.gnomview(maps_noiseless[i,:,s], reso=25, sub=(d['nf_recon'],3,3*i+s+1), 
                    title='Noiseless '+stn[s]+' Sub {}'.format(i))
        
plt.figure()
for i in range(d['nf_recon']):
    for s in range(3):
        hp.gnomview(maps_qubic[i,:,s], reso=25, sub=(d['nf_recon'],3,3*i+s+1), 
                    title='QUBIC Fast Sim '+stn[s]+' Sub {}'.format(i))        
        
plt.figure()
for i in range(d['nf_recon']):
    for s in range(3):
        hp.gnomview(maps_noise[i,:,s], reso=25, sub=(d['nf_recon'],3,3*i+s+1), 
                    title='QUBIC Noise Fast Sim '+stn[s]+' Sub {}'.format(i))            

### Multiple Noise realizations
Because in the above mode, most of the CPU time is spent on convolving the input sky to the relevant resolutions, once one has made one sky realization, it is faster to make many noise only realizations that can be added to the input CMB afterwards:

In [ ]:
newd = d.copy()
newd['nf_recon'] = 3

##### QubicSkySim instanciation
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, newd)

##### Getting FastSimulator output maps
new_maps_noise, coverage = Qubic_sky.get_partial_sky_maps_withnoise(noise_only=True, 
                                                                    spatial_noise=False)
for i in range(newd['nf_recon']):
    for s in range(3):
        hp.gnomview(new_maps_noise[i, :, s], 
                    reso=25, 
                    sub=(newd['nf_recon'],3,3*i+s+1), 
                    title='QUBIC Noise Fast Sim ' + stn[s] + f' Sub {i}')        


# Checking the noise properties of the FastSimulation:

Of course we focus here on the noise_only results of the FastSimulator.

In [ ]:
# !!!! Choose a number of bands for all the following
nbands = 7

newd = d.copy()
newd['nf_recon'] = nbands

seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, newd)

In [ ]:
### Let's read the input noise properties from the file that is used in the fast simulator 
### (these are fitted to end-to=-end simulations)
version_FastSim = '01'
rep = os.path.join(data_dir, f'FastSimulator_version{version_FastSim}')

pklfile = rep + os.sep + f'DataFastSimulator_{config}_nfsub_{nbands}.pkl'
print('reading pickle file: %s' % pklfile)

with open(pklfile, "rb") as file:
    DataFastSim = pickle.load(file)

print(DataFastSim.keys())
print(DataFastSim['signoise'])
print(DataFastSim['alpha'])

### 1) Noise Profile
This one cannot be checked if the keyword 'spatial_noise' is True in the call, we therefore make a new realization with this keyword set to False. Finally, in order to be comparable to the input simulations, one needs to 

In [ ]:
new_maps_noise, coverage = Qubic_sky.get_partial_sky_maps_withnoise(noise_only=True,
                                                                    spatial_noise=False,
                                                                    sigma_sec=100)

plt.figure()
for isub in range(newd['nf_recon']):
    sqn = int(np.sqrt(newd['nf_recon']))
    if (sqn**2) != newd['nf_recon']:
        sqn += 1
    plt.subplot(sqn, sqn, isub+1)
    xx, yyfs, fitcov = qss.get_noise_invcov_profile(new_maps_noise[isub, :, :], 
                                                    coverage, 
                                                    QUsep=True, 
                                                    nbins=30,
                                                    label='FastSim sub={}/{}'.format(isub+1, newd['nf_recon']), 
                                                    fit=False, 
                                                    norm=False,
                                                    allstokes=False, doplot=True)
    fitcov = DataFastSim['effective_variance_invcov']
    plt.plot(fitcov[isub][0,:], np.sqrt(fitcov[isub][1, :]), label='I from End-to-End')
    plt.plot(fitcov[isub][0,:], np.sqrt(fitcov[isub][2, :]), label='QU from End-to-End')
    plt.legend()
    plt.xlim(0.9, 10.1)
#     ylim(0, np.max(yyfs)*1.1)
    

### 2) SubBand-SubBand correlation:
For this we can use the above simulation

In [ ]:
new_maps_noise, coverage = Qubic_sky.get_partial_sky_maps_withnoise(noise_only=True, 
                                                                    spatial_noise=False)


cInew, cQnew, cUnew, fitcov, noise_norm = qss.get_cov_nunu(new_maps_noise, coverage, QUsep=True)
new_corr_mats = [cInew, cQnew/2, cUnew/2]

corr_mats = [DataFastSim['CovI'], DataFastSim['CovQ']/2, DataFastSim['CovU']/2]

plt.figure()
stn = ['I','Q/2', 'U/2']
# bla = np.max([np.abs(np.array([corr_mats, new_corr_mats])), np.array([corr_mats, new_corr_mats])])
mini = None
maxi = None
for s in range(3):
    plt.subplot(2,3,1+s)
    plt.imshow(corr_mats[s], vmin=mini, vmax=maxi, cmap='bwr')
    plt.colorbar(orientation='horizontal')
    plt.title('End-To-End Cov {} nsub={}'.format(stn[s],newd['nf_recon']))
    
    plt.subplot(2, 3, 4+s)
    plt.imshow(new_corr_mats[s], vmin=mini, vmax=maxi, cmap='bwr')
    plt.colorbar(orientation='horizontal')
    plt.title('FastSim Cov {} nsub={}'.format(stn[s],newd['nf_recon']))


### 3) Spatial correlation
One needs to turn spatial_noise to True obviously...

In [ ]:
new_maps_noise, coverage = Qubic_sky.get_partial_sky_maps_withnoise(noise_only=True, 
                                                                    spatial_noise=True)

In [ ]:
### Get the C(theta)
allcthI = []
allcthQ = []
allcthU = []
pixgood = (coverage/np.max(coverage)) > 0.1 
for i in range(newd['nf_recon']):
    corrected_qubicnoise = qss.correct_maps_rms(new_maps_noise[i,:,:], 
                                                coverage, 
                                                DataFastSim['effective_variance_invcov'][i])
    th, thecthI, _ = qss.ctheta_parts(corrected_qubicnoise[:, 0], pixgood, 0, 20, 20, nsplit=5, degrade_init=128)
    allcthI.append(thecthI)
    th, thecthQ, _ = qss.ctheta_parts(corrected_qubicnoise[:, 1], pixgood, 0, 20, 20, nsplit=5, degrade_init=128)
    allcthQ.append(thecthQ)
    th, thecthU, _ = qss.ctheta_parts(corrected_qubicnoise[:, 2], pixgood, 0, 20, 20, nsplit=5, degrade_init=128)
    allcthU.append(thecthU)
    print(i)

In [ ]:
for i in range(newd['nf_recon']):
    plt.plot(th, allcthI[i]/allcthI[i][0], 'o', label=f'Fast-Simulation I Sub {i+1}')
    plt.plot(th, allcthQ[i]/allcthQ[i][0], 'o', label=f'Fast-Simulation Q Sub {i+1}')
    plt.plot(th, allcthU[i]/allcthU[i][0], 'o', label=f'Fast-Simulation U Sub {i+1}')
    
plt.legend()
plt.xlabel(r'$\theta$ [deg]')
plt.ylabel(r'$C(\theta$)')

### 4) RMS profile

In [ ]:
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, newd)

new_maps_noise, coverage = Qubic_sky.get_partial_sky_maps_withnoise(noise_only=True,
                                                                    spatial_noise=False)

xx, yyI, yyQ, yyU = qss.get_angular_profile(new_maps_noise[0, :, :],
                                                nbins=30,
                                                separate=True,
                                                center=[0, 0])
pix_size = hp.nside2resol(256, arcmin=True)
meanvalI = np.mean(yyI[xx < 10]) * pix_size
meanvalQU = np.mean((yyQ[xx < 10] + yyQ[xx < 10]) / 2) * pix_size

plt.figure()
plt.plot(xx, yyI * pix_size, 'o', label='I')
plt.plot(xx, yyQ * pix_size, 'o', label='Q')
plt.plot(xx, yyU * pix_size, 'o', label='U')

plt.axhline(y=meanvalI,
            label=r'I RMS = {0:5.1f} $\mu K.arcmin$'.format(meanvalI),
            color='r', ls=':')
plt.axhline(y=meanvalQU,
            label=r'QU RMS = {0:5.1f} $\mu K.arcmin$'.format(meanvalQU),
            color='m', ls=':')

plt.xlabel('Degrees from center of the field')
plt.ylabel(r'Noise RMS $[\mu K.arcmin]$')
plt.title('QUBIC FastSim - ' + config + ' - Nptg = 10000')
plt.legend(fontsize=11)
plt.xlim(0, 20)
plt.ylim(0, meanvalQU * 2)